In [ ]:
EPSILON = 1e-10

import numpy as np
from sklearn.metrics import mean_squared_error


def _percentage_error(actual: np.ndarray, predicted: np.ndarray):
    """
    Percentage error
    Note: result is NOT multiplied by 100
    """
    return _error(actual, predicted) / (actual + EPSILON)


def _error(actual: np.ndarray, predicted: np.ndarray):
    """ Simple error """
    return actual - predicted


def mae(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Absolute Error """
    return np.mean(np.abs(_error(actual, predicted)))


def _naive_forecasting(actual: np.ndarray, seasonality: int = 1):
    """ Naive forecasting method which just repeats previous samples """
    return actual[:-seasonality]


def mase(actual: np.ndarray, predicted: np.ndarray, seasonality: int = 1):
    """
    Mean Absolute Scaled Error
    Baseline (benchmark) is computed with naive forecasting (shifted by @seasonality)
    """
    return mae(actual, predicted) / mae(actual[seasonality:], _naive_forecasting(actual, seasonality))



def normalize_data(df):
    """
    Normalizes the data to number between [-1, 1] inclusive.

    :param df: data to normalize
    :return: the newly scaled data
    """
    x_train = df.values
    scaler = MinMaxScaler()
    x_scaled = scaler.fit_transform(x_train)
    return pd.DataFrame(x_scaled, columns=df.columns)


def chunk_data(data, chunk_size):
    """
    function chunks the data into several equally sized data chunks, all of them meant to
    be inputted into the LSTM as such. This is a point to messing around to test performance
    to see if smaller or larger chunk sizes get better performance.

    :param data: data input, mean to be numpy array from dataFrame.values
    :param chunk_size: number of samples to take from the data given
    :return: chunked data
    """
    balancer = data.shape[0] % chunk_size
    if balancer == 0:
        return array(split(data, len(data)/chunk_size))
    else:
        # needed since all arrays must be balanced for number of features
        # This removes the extra features that would result in an uneven balance on the last array
        # only removes the final elements
        removal_list = []
        for i in range(balancer, 0, -1):
            removal_list.append(data.shape[0] - i)

        data = np.delete(data, removal_list, axis=0)
        return array(split(data, len(data) / chunk_size))


def forecast_evals(forecasted: np.ndarray, actual: np.ndarray):
    """
    Calculates several different forecast evaluation methods.

    :param forecasted: data predicted using the forecast model
    :param actual: actual value excepted
    :return:
    """

    try:
        me = np.mean(forecasted - actual)  # mean error
    except Exception as err:
        me = 'Unable to compute metric {0}'.format(err)

    try:
        mae_ = np.mean(np.abs(forecasted - actual))  # mean absolute error
    except Exception as err:
        mae_ = 'Unable to compute metric {0}'.format(err)

    try:
        mse = mean_squared_error(forecasted, actual)  # mean squared error
    except Exception as err:
        mse = 'Unable to compute metric {0}'.format(err)

    try:
        rmse = np.mean((forecasted - actual) ** 2) ** .5  # Root Mean Squared Error
    except Exception as err:
        rmse = 'Unable to compute metric {0}'.format(err)

    try:
        corr = np.corrcoef(forecasted, actual)[0, 1]
    except Exception as err:
        corr = 'Unable to compute metric {0}'.format(err)

    try:
        over_expect = sum(forecasted > actual) / len(actual)  # percentage that forecasted
    except Exception as err:
        over_expect = 'Unable to compute metric {0}'.format(err)
    # over predicts the actual

    try:
        residuals = forecasted - actual
    except Exception as err:
        residuals = 'Unable to compute metric {0}'.format(err)

    try:
        mfe = np.mean(residuals)  # mean of the residuals
    except Exception as err:
        mfe = 'Unable to compute metric {0}'.format(err)

    try:
        mase_ = mase(actual, forecasted)
    except Exception as err:
        mase_ = 'Unable to compute metric {0}'.format(err)

    return residuals, ({'me': me, 'mae': mae_, "mse": mse,
              'rmse': rmse, "over_shot": over_expect, "mfe": mfe,
             'corr': corr, "mase": mase_})
